# Imports

In [1]:
import os
import sys
import glob
import datetime

import pandas as pd

In [2]:
sys.path.append(os.path.abspath('../../'))
sys.path.append(os.path.abspath('../../eventpredictor'))

import eventpredictor.config as paths
from eventpredictor.data.download_data import FootballScraper as fs

2024-06-02 12:46:41.606 | INFO     | config:<module>:11 - PROJ_ROOT path is: /Users/maurycystopowski/Documents/Stuff/Projects/eventpredictor
2024-06-02 12:46:41.622 | INFO     | eventpredictor.config:<module>:11 - PROJ_ROOT path is: /Users/maurycystopowski/Documents/Stuff/Projects/eventpredictor


In [7]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)

In [4]:
df = pd.read_csv(os.path.join(paths.INCIDENTS, 'incidents.csv'))

In [5]:
df.head()

,hy_1,hy_2,hr_1,hr_2,hyr_1,hyr_2,hg_1,hg_2,h_ht,h_ft,hp,hpm,hog,hs_1,hs_2,ay_1,ay_2,ar_1,ar_2,ayr_1,ayr_2,ag_1,ag_2,a_ht,a_ft,ap,apm,aog,as_1,as_2,id
0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,3.0,NaN,1.0,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,3.0,6768419
1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,3.0,NaN,2.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,3.0,7305717
2,1.0,2.0,NaN,NaN,NaN,NaN,1.0,2.0,1.0,3.0,1.0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,2.0,NaN,2.0,2.0,NaN,1.0,NaN,NaN,NaN,7404120
3,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,1.0,NaN,NaN,NaN,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,9595395
4,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,2.0,NaN,1.0,NaN,NaN,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,3.0,7861457


In [8]:
df.isna().sum()

hy_1      95265
hy_2      60635
hr_1     167931
hr_2     164919
hyr_1    169554
hyr_2    164603
hg_1      89158
hg_2      75359
h_ht      89182
h_ft      40638
hp       150238
hpm      164744
hog      165081
hs_1     153107
hs_2      22684
ay_1      87597
ay_2      55386
ar_1     167463
ar_2     164179
ayr_1    169208
ayr_2    163045
ag_1     103776
ag_2      90115
a_ht     103792
a_ft      56185
ap       155866
apm      166278
aog      165926
as_1     152775
as_2      22674
id            0
dtype: int64

# Missing goals

In [9]:
df.loc[(df['hg_1'].isna()) & (df['h_ht'].notna()), 'hg_1'] = df['h_ht']
df.loc[(df['hg_1'].notna()) & (df['h_ht'].isna()), 'h_ht'] = df['hg_1']

In [10]:
df.loc[(df['ag_1'].isna()) & (df['a_ht'].notna()), 'ag_1'] = df['a_ht']
df.loc[(df['ag_1'].notna()) & (df['a_ht'].isna()), 'a_ht'] = df['ag_1']

In [11]:
df.loc[(df['hg_2'].isna()) & (df['h_ft'].notna()) & (df['hg_1'].notna()), 'hg_2'] = df['h_ft'] - df['hg_1']
df.loc[(df['hg_2'].notna()) & (df['h_ft'].isna()) & (df['hg_1'].notna()), 'h_ft'] = df['hg_2'] + df['hg_1']

In [12]:
df.loc[(df['ag_2'].isna()) & (df['a_ft'].notna()) & (df['ag_1'].notna()), 'ag_2'] = df['a_ft'] - df['ag_1']
df.loc[(df['ag_2'].notna()) & (df['a_ft'].isna()) & (df['ag_1'].notna()), 'a_ft'] = df['ag_2'] + df['ag_1']

In [13]:
df.loc[(df['h_ft'].notna()) & (df['hg_2'].notna()) & (df['hg_1'].isna()), 'hg_1'] = df['h_ft'] - df['hg_2']
df.loc[(df['h_ft'].notna()) & (df['hg_2'].notna()) & (df['h_ht'].isna()), 'h_ht'] = df['h_ft'] - df['hg_2']

In [14]:
df.loc[(df['a_ft'].notna()) & (df['ag_2'].notna()) & (df['ag_1'].isna()), 'ag_1'] = df['a_ft'] - df['ag_2']
df.loc[(df['a_ft'].notna()) & (df['ag_2'].notna()) & (df['a_ht'].isna()), 'a_ht'] = df['a_ft'] - df['ag_2']

In [16]:
df.describe()

,hy_1,hy_2,hr_1,hr_2,hyr_1,hyr_2,hg_1,hg_2,h_ht,h_ft,hp,hpm,hog,hs_1,hs_2,ay_1,ay_2,ar_1,ar_2,ayr_1,ayr_2,ag_1,ag_2,a_ht,a_ft,ap,apm,aog,as_1,as_2,id
count,75307.000000,109937.000000,2641.000000,5653.000000,1018.000000,5969.000000,130076.000000,130187.000000,130076.000000,130128.000000,20334.000000,5828.000000,5491.000000,17465.000000,147888.000000,82975.000000,115186.000000,3109.000000,6393.000000,1364.000000,7527.000000,114474.000000,114642.000000,114474.000000,114510.000000,14706.000000,4294.000000,4646.000000,17797.000000,147898.000000,1.705720e+05
mean,1.361587,1.753304,1.024612,1.034849,1.004912,1.022449,0.857145,1.081475,0.857645,1.937615,1.054638,1.022135,1.015662,1.075637,3.318221,1.399506,1.823546,1.020264,1.036759,1.006598,1.024844,0.749865,0.969008,0.750170,1.717431,1.043248,1.016302,1.012914,1.073776,3.313716,8.577228e+06
std,0.623123,0.918598,0.159782,0.192818,0.069945,0.150397,0.836389,0.909019,0.837002,1.105046,0.233468,0.147134,0.124175,0.282839,1.080769,0.650086,0.958951,0.143189,0.197117,0.080991,0.158200,0.764892,0.836755,0.765329,0.958627,0.210647,0.126648,0.112917,0.279088,1.080629,1.404442e+06
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-2.000000,-1.000000,-2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.617268e+06
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,7.434438e+06
50%,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,3.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,8.254646e+06
75%,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,4.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,4.000000,9.627707e+06
max,8.000000,9.000000,3.000000,3.000000,2.000000,3.000000,16.000000,9.000000,16.000000,20.000000,3.000000,2.000000,2.000000,6.000000,10.000000,8.000000,10.000000,3.000000,3.000000,2.000000,3.000000,8.000000,9.000000,8.000000,13.000000,4.000000,2.000000,2.000000,5.000000,9.000000,1.202873e+07


In [15]:
df.isna().sum()

hy_1      95265
hy_2      60635
hr_1     167931
hr_2     164919
hyr_1    169554
hyr_2    164603
hg_1      40496
hg_2      40385
h_ht      40496
h_ft      40444
hp       150238
hpm      164744
hog      165081
hs_1     153107
hs_2      22684
ay_1      87597
ay_2      55386
ar_1     167463
ar_2     164179
ayr_1    169208
ayr_2    163045
ag_1      56098
ag_2      55930
a_ht      56098
a_ft      56062
ap       155866
apm      166278
aog      165926
as_1     152775
as_2      22674
id            0
dtype: int64

In [17]:
df.head()

,hy_1,hy_2,hr_1,hr_2,hyr_1,hyr_2,hg_1,hg_2,h_ht,h_ft,hp,hpm,hog,hs_1,hs_2,ay_1,ay_2,ar_1,ar_2,ayr_1,ayr_2,ag_1,ag_2,a_ht,a_ft,ap,apm,aog,as_1,as_2,id
0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,3.0,NaN,1.0,NaN,NaN,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,3.0,6768419
1,1.0,1.0,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,3.0,NaN,2.0,NaN,1.0,NaN,NaN,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,3.0,7305717
2,1.0,2.0,NaN,NaN,NaN,NaN,1.0,2.0,1.0,3.0,1.0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,2.0,0.0,2.0,2.0,NaN,1.0,NaN,NaN,NaN,7404120
3,1.0,1.0,NaN,NaN,NaN,NaN,1.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,9595395
4,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,2.0,NaN,1.0,NaN,NaN,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,3.0,7861457


In [18]:
df.to_csv(os.path.join(paths.INCIDENTS, 'incidents_with_goals_filled.csv'), index=False)

: 